In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Paso 1: Cargar los datos

In [ ]:
df = pd.read_csv('../data/teleCust1000t.csv')  # Reemplazar con la ruta de tu dataset
print("Dimensiones del dataset:", df.shape)
print("\nInformación del dataset:")
df.head()

# Paso 2: Resumen estadístico

In [ ]:
print("\nEstadísticas descriptivas:")
print(df.describe())

# Paso 3: Verificar valores faltantes y duplicados

In [ ]:
missing_values = df.isnull().sum()
print("\nValores faltantes por columna:")
print(missing_values)

duplicates = df.duplicated().sum()
print(f"\nRegistros duplicados: {duplicates}")

# Paso 4: Distribución de la variable objetivo

In [ ]:
plt.figure(figsize=(10, 6))
class_dist = df['custcat'].value_counts()
sns.barplot(x=class_dist.index, y=class_dist.values)
plt.title('Distribución de Categorías de Clientes')
plt.xlabel('Categoría')
plt.ylabel('Cantidad')
plt.show()

print("\nPorcentaje por categoría:")
print((class_dist / len(df) * 100).round(2))

# Paso 5: Análisis de variables numéricas

In [ ]:
numeric_cols = ['tenure', 'age', 'income', 'ed', 'employ', 'address', 'reside']
for col in numeric_cols:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Histograma
    sns.histplot(data=df, x=col, hue='custcat', kde=True, ax=ax1)
    ax1.set_title(f'Distribución de {col}')

    # Boxplot
    sns.boxplot(data=df, x='custcat', y=col, ax=ax2)
    ax2.set_title(f'Boxplot de {col} por Categoría')

    plt.tight_layout()
    plt.show()

# Paso 6: Análisis de variables categóricas

In [ ]:
cat_cols = ['region', 'marital', 'gender']
for col in cat_cols:
    plt.figure(figsize=(10, 6))
    crosstab = pd.crosstab(df[col], df['custcat'], normalize='index') * 100
    crosstab.plot(kind='bar', stacked=True)
    plt.title(f'Distribución de Categorías por {col}')
    plt.xlabel(col)
    plt.ylabel('Porcentaje')
    plt.legend(title='Categoría')
    plt.show()


# Paso 7: Matriz de correlación

In [ ]:
plt.figure(figsize=(12, 8))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación')
plt.show()

# Paso 8: Feature Engineering
>
Crear nuevas variables derivadas

In [ ]:
df['income_per_family'] = df['income'] / (df['reside'] + 1)
df['tenure_to_age_ratio'] = df['tenure'] / df['age']
df['income_per_education'] = df['income'] / df['ed']

# Segmentación
df['age_segment'] = pd.qcut(df['age'], q=5, labels=['Very Young', 'Young', 'Middle', 'Senior', 'Very Senior'])
df['income_segment'] = pd.qcut(df['income'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
df['tenure_segment'] = pd.qcut(df['tenure'], q=5, labels=['New', 'Recent', 'Established', 'Loyal', 'Very Loyal'])

# Interacciones
df['income_tenure_interaction'] = df['income'] * df['tenure']
df['age_income_ratio'] = df['age'] / df['income']

# Validación visual de las nuevas características
new_features = ['income_per_family', 'tenure_to_age_ratio', 'income_per_education',
                'income_tenure_interaction', 'age_income_ratio']
for col in new_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='custcat', y=col)
    plt.title(f'Distribución de {col} por Categoría')
    plt.show()

# Paso 9: Escalado de variables numéricas

In [ ]:
scaler = StandardScaler()
numeric_cols_to_scale = ['tenure', 'age', 'income', 'address', 'ed', 'employ', 'reside']
df[numeric_cols_to_scale] = scaler.fit_transform(df[numeric_cols_to_scale])

print("\nDataset procesado listo para modelado:")
print(df.head())

# Guardar el DataFrame procesado
df.to_csv('../data/proc_escalado.csv', index=False)
print("\nDataFrame procesado guardado")

# Paso 10: Modelado

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score

def create_polynomial_features(X, degree=2):
    # Seleccionar solo las características numéricas más importantes
    important_features = ['tenure_to_age_ratio', 'income_per_education',
                         'income_tenure_interaction', 'age_income_ratio',
                         'ed', 'tenure']

    # Crear características polinómicas
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    poly_features = poly.fit_transform(X[important_features])

    # Crear DataFrame con los nuevos nombres
    feature_names = poly.get_feature_names_out(important_features)
    poly_df = pd.DataFrame(poly_features, columns=feature_names)

    # Combinar con características originales
    X_new = X.copy()
    for col in poly_df.columns:
        if col not in X.columns:
            X_new[col] = poly_df[col]

    return X_new

def prepare_data():
    # Cargar datos
    df = pd.read_csv('../data/proc_escalado.csv')
    X = df.drop(['custcat'], axis=1).values
    y = df['custcat'].values - 1  # Ajustar etiquetas para empezar desde 0

    # División de datos
    X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.2, random_state=42)

    # Estandarización
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    # Crear datasets
    train_dataset = CustomDataset(X_train, y_train)
    val_dataset = CustomDataset(X_val, y_val)
    test_dataset = CustomDataset(X_test, y_test)

    return train_dataset, val_dataset, test_dataset, X_train.shape[1]

def train_xgboost(X_train, X_test, y_train, y_test):
    # Parámetros más extensos para XGBoost
    param_grid = {
        'n_estimators': [200, 300, 400],
        'max_depth': [4, 5, 6],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'min_child_weight': [1, 3, 5]
    }

    # Crear y entrenar modelo
    model = XGBClassifier(random_state=42)
    grid = GridSearchCV(
        model,
        param_grid,
        cv=5,
        n_jobs=-1,
        scoring='accuracy',
        verbose=1
    )

    print("Entrenando XGBoost con parámetros mejorados...")
    grid.fit(X_train, y_train)

    # Imprimir resultados
    print("\nMejores parámetros:")
    print(grid.best_params_)
    print(f"Precisión en validación: {grid.best_score_:.4f}")
    print(f"Precisión en test: {grid.score(X_test, y_test):.4f}")

    # Evaluar en conjunto de prueba
    y_pred = grid.predict(X_test)
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred,
                              target_names=['Clase 1', 'Clase 2', 'Clase 3', 'Clase 4']))

    # Matriz de confusión
    print("\nMatriz de confusión:")
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(conf_matrix)

    # Importancia de características
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': grid.best_estimator_.feature_importances_
    })
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    print("\nTop 15 características más importantes:")
    print(feature_importance.head(15))

    return grid.best_estimator_, feature_importance

def main():
    print("Cargando datos...")
    df = pd.read_csv('../data/proc_escalado.csv')

    print("\nPreparando datos con características mejoradas...")
    X_train, X_test, y_train, y_test = prepare_data(df)

    print("\nDimensiones de los datos:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"y_train: {y_train.shape}")
    print(f"y_test: {y_test.shape}")

    # Entrenar modelo mejorado
    best_model, feature_importance = train_xgboost(X_train, X_test, y_train, y_test)

if __name__ == "__main__":
    main()

Cargando datos...

Preparando datos con características mejoradas...

Dimensiones de los datos:
X_train: (900, 40)
X_test: (200, 40)
y_train: (900,)
y_test: (200,)
Entrenando XGBoost con parámetros mejorados...
Fitting 5 folds for each of 729 candidates, totalling 3645 fits

Mejores parámetros:
{'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 400, 'subsample': 0.8}
Precisión en validación: 0.4467
Precisión en test: 0.4050

Reporte de clasificación:
              precision    recall  f1-score   support

     Clase 1       0.40      0.36      0.38        53
     Clase 2       0.36      0.39      0.37        44
     Clase 3       0.45      0.46      0.46        56
     Clase 4       0.40      0.40      0.40        47

    accuracy                           0.41       200
   macro avg       0.40      0.40      0.40       200
weighted avg       0.40      0.41      0.40       200


Matriz de confusión:
[[19 11 15  8]
 [ 8 17  9 10]
 [14 

# 11 Red Neuronal

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import optuna

# 1. Carga y preprocesamiento de datos
def prepare_data():
    # Cargar el dataset
    df = pd.read_csv('../data/proc_escalado.csv')

    # Variables categóricas a codificar
    categorical_cols = ['region', 'marital', 'gender', 'age_segment',
                       'income_segment', 'tenure_segment']

    # Codificar variables categóricas
    label_encoders = {}
    for col in categorical_cols:
        label_encoders[col] = LabelEncoder()
        df[col] = label_encoders[col].fit_transform(df[col])

    # Separar características y variable objetivo
    X = df.drop('custcat', axis=1)
    y = df['custcat'].values - 1  # Ajustar etiquetas para empezar desde 0

    # Escalar características numéricas
    numeric_cols = ['tenure', 'age', 'income', 'ed', 'employ', 'address', 'reside',
                   'income_per_family', 'tenure_to_age_ratio', 'income_per_education',
                   'income_tenure_interaction', 'age_income_ratio']

    scaler = StandardScaler()
    X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

    return X, y

# 2. Definición del modelo con bloques residuales
class ResidualBlock(layers.Layer):
    def __init__(self, units, dropout_rate):
        super(ResidualBlock, self).__init__()
        self.dense1 = layers.Dense(units, activation='relu')
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dense2 = layers.Dense(units, activation='relu')
        self.dropout2 = layers.Dropout(dropout_rate)
        self.add = layers.Add()

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.dropout2(x)

        # Asegurar que las dimensiones coincidan para la conexión residual
        if inputs.shape[-1] != x.shape[-1]:
            inputs = layers.Dense(x.shape[-1])(inputs)

        return self.add([x, inputs])

def create_model(num_blocks, units, dropout_rate, learning_rate, input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(units)(inputs)

    for _ in range(num_blocks):
        x = ResidualBlock(units, dropout_rate)(x)

    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(4, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# 3. Función objetivo para Optuna
def objective(trial):
    # Hiperparámetros a optimizar
    num_blocks = trial.suggest_int('num_blocks', 10, 30)
    units = trial.suggest_int('units', 128, 512)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    decay_rate = trial.suggest_float('decay_rate', 0.9, 0.99)

    # Learning rate scheduler
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        learning_rate, decay_steps=1000, decay_rate=decay_rate
    )

    # Callbacks
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )

    # Crear modelo
    model = create_model(
        num_blocks=num_blocks,
        units=units,
        dropout_rate=dropout_rate,
        learning_rate=lr_schedule,
        input_shape=(X.shape[1],)
    )

    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []

    for train_index, val_index in skf.split(X, y):
        X_train_fold = X.iloc[train_index].values
        X_val_fold = X.iloc[val_index].values
        y_train_fold = y[train_index]
        y_val_fold = y[val_index]

        history = model.fit(
            X_train_fold, y_train_fold,
            epochs=50,
            batch_size=32,
            validation_data=(X_val_fold, y_val_fold),
            callbacks=[early_stopping],
            verbose=0
        )

        cv_scores.append(max(history.history['val_accuracy']))

    return np.mean(cv_scores)

# 4. Ejecución principal
if __name__ == "__main__":
    # Preparar datos
    print("Cargando y preparando datos...")
    X, y = prepare_data()
    print(f"Dimensiones del dataset: {X.shape}")

    # Configurar y ejecutar la optimización
    print("Iniciando optimización con Optuna...")
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100, timeout=7200)  # 2 horas máximo

    # Imprimir resultados
    print("\nMejores hiperparámetros encontrados:")
    print(study.best_params)
    print(f"Mejor precisión: {study.best_value:.4f}")

    # Entrenar modelo final con los mejores hiperparámetros
    print("\nEntrenando modelo final con los mejores hiperparámetros...")
    final_model = create_model(
        num_blocks=study.best_params['num_blocks'],
        units=study.best_params['units'],
        dropout_rate=study.best_params['dropout_rate'],
        learning_rate=study.best_params['learning_rate'],
        input_shape=(X.shape[1],)
    )

    # Dividir datos para evaluación final
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar modelo final
    final_model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
    )

    # Evaluar modelo final
    test_loss, test_accuracy = final_model.evaluate(X_test, y_test)
    print(f"\nPrecisión final en conjunto de prueba: {test_accuracy:.4f}")

    # Guardar modelo
    final_model.save('final_model')


[I 2024-12-05 20:29:21,864] A new study created in memory with name: no-name-e74f57ab-53a3-4058-a933-96254686093a


Cargando y preparando datos...
Dimensiones del dataset: (1000, 19)
Iniciando optimización con Optuna...



[I 2024-12-05 20:29:55,957] Trial 0 finished with value: 0.2810000002384186 and parameters: {'num_blocks': 26, 'units': 193, 'learning_rate': 3.7920210661803664e-05, 'dropout_rate': 0.49757440993932467, 'decay_rate': 0.9464806346408275}. Best is trial 0 with value: 0.2810000002384186.
[I 2024-12-05 20:30:17,882] Trial 1 finished with value: 0.23699999749660491 and parameters: {'num_blocks': 10, 'units': 289, 'learning_rate': 1.2014718671334378e-05, 'dropout_rate': 0.4320407262126319, 'decay_rate': 0.9688970220979977}. Best is trial 0 with value: 0.2810000002384186.
[I 2024-12-05 20:30:44,942] Trial 2 finished with value: 0.4 and parameters: {'num_blocks': 14, 'units': 233, 'learning_rate': 6.280677374431575e-05, 'dropout_rate': 0.17540532997230446, 'decay_rate': 0.9592186338416457}. Best is trial 2 with value: 0.4.
[I 2024-12-05 20:31:12,012] Trial 3 finished with value: 0.2539999961853027 and parameters: {'num_blocks': 13, 'units': 406, 'learning_rate': 2.7654590982977944e-05, 'dropou


Mejores hiperparámetros encontrados:
{'num_blocks': 14, 'units': 135, 'learning_rate': 0.00023973697573604274, 'dropout_rate': 0.10073826764345815, 'decay_rate': 0.9106855586085454}
Mejor precisión: 0.4660

Entrenando modelo final con los mejores hiperparámetros...
Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.3081 - loss: 1.9846 - val_accuracy: 0.2350 - val_loss: 4.8512
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.3573 - loss: 1.5562 - val_accuracy: 0.3850 - val_loss: 1.5289
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.3669 - loss: 1.4572 - val_accuracy: 0.3700 - val_loss: 1.4357
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.3722 - loss: 1.4503 - val_accuracy: 0.4350 - val_loss: 1.3045
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3892 - loss: 1.3432 - val_accuracy: 0.4150 - val_loss: 1.3036
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3931 - loss: 1.2970 - val_

In [66]:
import optuna
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report
import datetime

def prepare_data():
    try:
        # Cargar datos
        df = pd.read_csv('../data/proc_escalado.csv')
        print(f"Dimensiones del dataset: {df.shape}")

        # Identificar columnas no numéricas
        object_columns = df.select_dtypes(include=['object']).columns

        # Convertir columnas object a numéricas
        le = LabelEncoder()
        for col in object_columns:
            df[col] = le.fit_transform(df[col].astype(str))

        # Verificar que todas las columnas sean numéricas
        assert df.select_dtypes(include=['object']).empty, "Aún hay columnas no numéricas"

        # Separar features y target
        X = df.drop('custcat', axis=1)
        y = df['custcat'] - 1

        # Convertir a numpy arrays
        X = X.values.astype('float32')
        y = y.values.astype('int32')

        return X, y

    except Exception as e:
        print(f"Error en prepare_data: {str(e)}")
        raise

def create_model(trial, input_shape):
    # Hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    n_layers = trial.suggest_int('n_layers', 2, 5)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    units = trial.suggest_int('units', 32, 512)

    model = Sequential()

    # Primera capa
    model.add(Dense(units, activation='relu', input_shape=(input_shape,)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))

    # Capas ocultas
    for i in range(n_layers):
        units = units // 2
        if units < 32:
            units = 32
        model.add(Dense(units, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

    # Capa de salida
    model.add(Dense(4, activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def objective(trial):
    try:
        # Hiperparámetros
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

        # Crear modelo
        model = create_model(trial, X_train.shape[1])

        # Entrenar
        history = model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=50,
            validation_data=(X_val, y_val),
            callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
            verbose=0
        )

        return max(history.history['val_accuracy'])

    except Exception as e:
        print(f"Error en objective: {str(e)}")
        raise

# Crear directorio para modelos
os.makedirs('../models', exist_ok=True)

# Cargar y preparar datos
print("Cargando y preparando datos...")
X, y = prepare_data()

# Split de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Optimización con Optuna
print("Iniciando optimización con Optuna...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=7200)

print("\nMejores hiperparámetros encontrados:")
print(study.best_params)
print(f"\nMejor accuracy: {study.best_value:.4f}")

# Entrenar modelo final con los mejores hiperparámetros
print("\nEntrenando modelo final...")
final_model = create_model(study.best_trial, X_train.shape[1])

# Timestamp para los archivos
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Callbacks para el modelo final
final_callbacks = [
    EarlyStopping(patience=15, restore_best_weights=True),
    ModelCheckpoint(
        filepath=f'models/best_model_{timestamp}.keras',
        save_best_only=True,
        monitor='val_accuracy'
    )
]

# Entrenar modelo final
history = final_model.fit(
    X_train, y_train,
    batch_size=study.best_params['batch_size'],
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=final_callbacks,
    verbose=1
)

# Guardar modelo final
final_model.save(f'../models/final_model_{timestamp}.keras')

# Evaluar modelo final
y_pred = np.argmax(final_model.predict(X_test), axis=1)
print("\nReporte de clasificación final:")
print(classification_report(y_test, y_pred))

# Guardar resultados
results = {
    'best_params': study.best_params,
    'best_value': study.best_value,
    'history': history.history,
    'classification_report': classification_report(y_test, y_pred, output_dict=True)
}

pd.to_pickle(results, f'../models/results_{timestamp}.pkl')
print(f"\nResultados guardados en: ../models/results_{timestamp}.pkl")

[I 2024-12-06 09:39:40,275] A new study created in memory with name: no-name-76531741-cb49-4b79-bc43-9f402191a044


Cargando y preparando datos...
Dimensiones del dataset: (1000, 20)
Iniciando optimización con Optuna...


[I 2024-12-06 09:42:13,709] Trial 0 finished with value: 0.3812499940395355 and parameters: {'batch_size': 128, 'learning_rate': 0.00016176427878897555, 'n_layers': 2, 'dropout': 0.1099035639810079, 'units': 483}. Best is trial 0 with value: 0.3812499940395355.
[I 2024-12-06 09:42:25,545] Trial 1 finished with value: 0.3187499940395355 and parameters: {'batch_size': 64, 'learning_rate': 0.00018252338271454872, 'n_layers': 4, 'dropout': 0.3904442410997403, 'units': 335}. Best is trial 0 with value: 0.3812499940395355.
[I 2024-12-06 09:42:33,761] Trial 2 finished with value: 0.375 and parameters: {'batch_size': 128, 'learning_rate': 0.00886271435173204, 'n_layers': 2, 'dropout': 0.26752506633539785, 'units': 105}. Best is trial 0 with value: 0.3812499940395355.
[I 2024-12-06 09:42:44,952] Trial 3 finished with value: 0.3687500059604645 and parameters: {'batch_size': 128, 'learning_rate': 0.006619927547314162, 'n_layers': 2, 'dropout': 0.10577952380999173, 'units': 474}. Best is trial 0 w


Mejores hiperparámetros encontrados:
{'batch_size': 32, 'learning_rate': 0.00010069229805765904, 'n_layers': 2, 'dropout': 0.18428679794249464, 'units': 442}

Mejor accuracy: 0.4500

Entrenando modelo final...
Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.3025 - loss: 2.0346 - val_accuracy: 0.3063 - val_loss: 1.6157
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2750 - loss: 2.1018 - val_accuracy: 0.3000 - val_loss: 1.3749
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2914 - loss: 1.8666 - val_accuracy: 0.2438 - val_loss: 1.3905
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3458 - loss: 1.7463 - val_accuracy: 0.2625 - val_loss: 1.4617
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2985 - loss: 1.7598 - val_accuracy: 0.2500 - val_loss: 1.4637
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2836 - loss: 1.7735 - val_accuracy: 0.2500 - val_loss: 1.4631
Epoch 7/100
20/20 ━━━━━━━━